In [2]:
#pip install --upgrade selenium==4.10.0

### 키워드에 해당하는 블로그의 링크들을 데이터 프레임으로 추출

In [1]:
import os
import sys
import urllib.request
import json
import pandas as pd

def getresult(client_id,client_secret,query,display=10,start=1,sort='sim'):
    encText = urllib.parse.quote(query)
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText + \
    "&display=" + str(display) + "&start=" + str(start) + "&sort=" + sort

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        response_json = json.loads(response_body)
    else:
        print("Error Code:" + rescode)

    return pd.DataFrame(response_json['items'])['link']

In [7]:
chungju = ['청남대','대청댐전망대','청주동물원','수암골벽화마을','상당산성','청주랜드']
result_all=pd.DataFrame()

In [8]:
for query in chungju:
    client_id = 'frPEZvEmZbkhQTJR4a9u' 
    client_secret = '2nOa73V0gW' 
    #query = '동해%2B여행'
    display=100
    start=1
    sort='sim'
    #result_all=pd.DataFrame()

    for i in range(0,10):
        start= 1 + 100*i
        result= getresult(client_id,client_secret,query,display,start,sort)
        result_all=pd.concat([result_all,result])

In [9]:
# 네이버 블로그에 해당하는 링크만 추출
# 네이버 블로그가 아니면 발생하는 오류들이 존재했었음.

result_all = result_all[result_all[0].str.contains('naver')]

In [10]:
result_all.reset_index(drop=True,inplace=True)

In [11]:
result_all.shape

(5907, 1)

In [12]:
# 중복값 제거

result_all.drop_duplicates(inplace=True)

In [13]:
result_all = result_all.reset_index()
result_all = result_all.drop(columns='index')

In [14]:
result_all.shape

(5619, 1)

### 데이터 링크들에 각각 반복문으로 접속해서 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm
from selenium.webdriver.chrome.service import Service

service = Service(executable_path='"/Users/hyungson/Desktop/vincent/3-1/textdata/chromedriver-mac-arm64/chromedriver"')
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # GUI 없이 실행하려면 주석 해제
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=service, options=options)

titles = []
contents = []
links = []
total_results = len(result_all)

for i in tqdm(range(total_results)):
    link = result_all[0][i]
    if 'blog.naver.com' in link:
        driver.get(result_all[0][i])
        driver.implicitly_wait(2)  # time sleep
        iframe = driver.find_element(By.ID, 'mainFrame')  # 수정: By.ID 사용
        driver.switch_to.frame(iframe)
        try:
            content = driver.find_element(By.CSS_SELECTOR, 'div.se-main-container').text  # 본문 구좌
            content = content.replace('\n', '')  # 줄바꿈 제거
            contents.append(content)
            title_tag = driver.find_element(By.TAG_NAME, 'title')  # 태그로 타이틀을 검색
            title = title_tag.get_attribute('textContent')
            titles.append(title)
            links.append(link)
        # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
        except NoSuchElementException:
            pass

driver.quit()


Incompatible release of chromedriver (version 111.0.5563.64) detected in PATH: /opt/homebrew/bin/chromedriver
 38%|█████████████▉                       | 1905/5054 [54:33<2:18:18,  2.64s/it]

In [16]:
# 블로그 링크 별 제목, 본문, 링크를 추출해 데이터프레임으로 만듬.

import pandas as pd
blog_crawl1 = pd.DataFrame({'title':titles,'content':contents,'link':links})
blog_crawl1

,title,content,link
0,"청주 가볼만한곳 청남대 : 과거 대통령 별장, 현재는 대통령기념관 및 산책코스 ( ...","청주 가볼만한곳 청남대과거 대통령 별장,현재는 대통령기념관 및 산책코스(+ 입장료)...",https://blog.naver.com/ellasha123/223169599078
1,따뜻한 남쪽의 청남대/대통령 기념관 : 네이버 블로그,청남대/대통령 기념관충북 청주시 상당구 문의면 청남대길 646청남대 본관은 1993...,https://blog.naver.com/parkbg0714/223160811307
2,"청남대 봄기운 가득, 축제로 흥겨움 가득 : 네이버 블로그",지난 주말 영춘제가 열리고 있는 청남대를 다녀왔습니다. 영춘제는 봄을 맞이한다는 의...,https://blog.naver.com/alice8565/223088318794
3,청주 청남대 맛집 부부농장 : 네이버 블로그,안냐떼어 붕어에요여기 제가 엄청 어렸을 때 비닐하우스? 같은데서먹은 기억이 있는데 ...,https://blog.naver.com/shmwlove6/223066757792
4,청주 아이와 가볼만한곳-청남대 : 네이버 블로그,청남대는 대전근처에서 가까운 곳에 위치해 있어서 당일로 아이와 함께 다녀오기 좋은곳...,https://blog.naver.com/happymomtour/223138300353
...,...,...,...
540,"'봄을 봄, 희망을 봄' 2021 청남대 영춘제 : 네이버 블로그",봄기운이 완연해지면서 곳곳에 꽃의 향연이 펼쳐지며청남대에서는 영춘제 축제가 개최되었...,https://blog.naver.com/cjcityblog/222347671826
541,"(청주, 대전근교/여행/가볼만한 곳) 청남대 : 네이버 블로그",대전에 갈 일이 있을 때마다주변에 가볼만한 곳을 찾아본다.수년 전 엄마가 청남대를 ...,https://blog.naver.com/shbaeck/222383683409
542,청남대 대통령별장 : 네이버 블로그,4월의 봄날치고는몹시도 짓궂게 굴던 어제이른 아침 서울을 출발해서 11쯤청남대에 도...,https://blog.naver.com/indang61/40186881071
543,청주방향 문의청남대 휴게소 먹거리 간식 커피 / 신전떡볶이 비추천 : 네이버 블로그,안녕하세요 나나입니다.경기도로 휴가 가는 길에 휴게소를 한군데 들렀어요.이름도 생소...,https://blog.naver.com/hnk1280/223170344158


In [64]:
# 전처리 전, 1차 크롤링 결과 저장

blog_crawl.to_csv('blog_keyword_chungju.csv', index = False, encoding='utf-8-sig')

### 전처리 (함수로 한번에)

In [65]:
from konlpy.tag import Okt
import pykospacing
import kss
import re

In [66]:
import copy

with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.replace('\n','') for x in stopwords] # stopword 파일의 줄바꿈 문자 제거
okt = Okt()

def preprocess_korean(text):
    my_text = copy.copy(text)
    my_text = my_text.replace('\n', '')
    spacer = pykospacing.Spacing()
    my_text = spacer(my_text) #  띄어쓰기 교정
    sents = kss.split_sentences(my_text)#  sentence tokenization
    
    p = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]')
    results = []
    for sent in sents:
        result = []
        tokens = okt.morphs(sent, stem=True) # word tokenization
        for token in tokens:
            token = p.sub('', token) #  특수문자 제거
            if token not in stopwords:
                result.append(token) #  stopword removal
        results.extend(result)
    result = ' '.join(results)
    
    return result


In [67]:
import tqdm
from tqdm import trange, notebook, tqdm_pandas, tqdm

In [68]:
tmp_data = pd.DataFrame(blog_crawl.loc[:,'content'])

In [69]:
# 길이 5000개 이상은 시간소모를 야기하는 듯해서 삭제

index = []
for i in range(len(blog_crawl)):
    if (len(blog_crawl['content'][i])) >= 6000:
        print(i)
        index.append(i)

715
1268
2321
2348
2378
2383
2388
2439
2447
2662
3226
3237
3270
3417
3453
3469
3485


In [70]:
tmp_data.drop(index = index, inplace = True)

In [71]:
tmp_data.reset_index(inplace=True)

In [72]:
%%time
tqdm.pandas()
tmp_data['preprocessed_content'] = tmp_data['content'].progress_apply(lambda x: preprocess_korean(x))


100%|███████████████████████████████████████| 3486/3486 [23:01<00:00,  2.52it/s]

CPU times: user 21min 1s, sys: 1min 46s, total: 22min 47s
Wall time: 23min 1s


In [73]:
tmp_data = tmp_data.drop(columns='index')

In [74]:
tmp_data

,content,preprocessed_content
0,경기도 아이와 가볼만한곳궁평항 물놀이 주말 방문할아버지 동물농장 갔다가 코스로 많이...,경기도 가다 볼 만 한 곳 궁평항 물놀이 주말 방문 하다 아버지 동물농장 가다 코스...
1,경기도 화성 궁평항낚시체험 물놀이터글/사진 미니미엄빠요란했던 장맛비도 주춤하고 아이...,경기도 화성 궁평항 낚시 체험 물 놀이터 글 사진 미니 미엄 빠 요란하다 장맛 비...
2,또 오랜만에 올려보는 포스팅어제 다녀왔는데 만족스러워서 후다닥 써본다-!원래 울애기...,오랜 만 올리다 보다 포스팅 어제 다녀오다 만족스럽다 후 다닥 써다 보다 원래 울...
3,이번여름은 아들과함께 물놀이터 도장깨기를 해볼까했는데 첫번째는 광교호수공원 분수 물...,여름 은 아들 물 놀이터 도장 깨다 해보다 하다 첫 번 째 는 광교 호수 공원 분수...
4,"궁평항 낚시화성 아이와가볼만한곳©글,사진 준브로맘안녕하세요? 준브로맘이예요~^^지난...",궁평항 낚시 화성 가보다 만 한 곳 글 사진 준 브로 맘 안녕하다 준 브로 맘...
...,...,...
3481,화성 용주사 효행박물관 용주사 상량문 –경기 화성 [36]용주사 입구 좌측에 있는 ...,화성 용주사 효행 박물관 용주사 상량문 경기 화성 용주사 입구 좌측 효행 박...
3482,효찰대본산 용주사대한불교조계종 제2교구 본사주소 / 경기도 화성시 송산동 187-2...,효 차다 본 산 용주사 대 한 불교 조계종 교구 본사 주소 경기도 화성시 송산동...
3483,응?나는 확인했다! 월요일 휴관 ㅋㅋ공휴일과 정기 휴관일(월요일)이 겹치는 경우당일...,는 확인 하다 월요일 관 ㅋㅋ 공휴일 정기 관일 월요일 겹 치다 경우 당일 ...
3484,"경기도 가볼만한 곳, 화성 용주사에서 특별했던 경험(ft. 대웅보전)경기도 화성 1...",경기도 가다 볼 만 한 곳 화성 용주사 특별하다 경험 대웅보전 경기도 화성...


In [75]:
tmp_data.to_csv('cluster_data/preprocessed_blog_chungju.csv', index = False, encoding='utf-8-sig')